夏目漱石の小説『吾輩は猫である』の文章（neko.txt）をCaboChaを使って係り受け解析し，その結果をneko.txt.cabochaというファイルに保存せよ．このファイルを用いて，以下の問に対応するプログラムを実装せよ．

In [1]:
! cabocha -f1 neko.txt > neko.txt.cabocha

---

### 40. 係り受け解析結果の読み込み（形態素）
形態素を表すクラスMorphを実装せよ．このクラスは表層形（surface），基本形（base），品詞（pos），品詞細分類1（pos1）をメンバ変数に持つこととする．さらに，CaboChaの解析結果（neko.txt.cabocha）を読み込み，各文をMorphオブジェクトのリストとして表現し，3文目の形態素列を表示せよ．

In [24]:
import re

class Morph:    
    def __init__(self, surface, pos, pos1, base):
        self.surface = surface
        self.base = base
        self.pos = pos
        self.pos1 = pos1
    
    def __str__(self):
        return ("Morph: {\"surface\": \"%s\", \"base\": \"%s\" ,\"pos\": \"%s\", \"pos1\": \"%s\"}" %
                (self.surface, self.base, self.pos, self.pos1))

    def __repr__(self):
        return self.__str__()

def make_morph(word):
    splitted = re.split('[\t,]', word)
    return Morph(splitted[0], splitted[7], splitted[1], splitted[2])
    
def neko_morphed():
    with open("neko.txt.cabocha", "rt") as neko:
        return [[make_morph(word) for word in line.split('\n')
                 if len(word) >= 4 and not re.match(r'^\*.*', word)]
            for line in re.split(r'EOS\n', neko.read())
                if len(line) is not 0]

print(neko_morphed()[2])

[Morph: {"surface": "名前", "base": "一般" ,"pos": "名前", "pos1": "名詞"}, Morph: {"surface": "は", "base": "係助詞" ,"pos": "は", "pos1": "助詞"}, Morph: {"surface": "まだ", "base": "助詞類接続" ,"pos": "まだ", "pos1": "副詞"}, Morph: {"surface": "無い", "base": "自立" ,"pos": "無い", "pos1": "形容詞"}, Morph: {"surface": "。", "base": "句点" ,"pos": "。", "pos1": "記号"}]
